In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/news_tagging_model
# !mkdir news_tagging_model

In [ ]:
!ls

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
# import pandas as pd
# df = pd.read_csv("univision_2015-2022_preprocessed.csv")
# df.category.value_counts()
# df_health = df[df['is_politics'] == 1]
# df_health
# df_not_health = df[df['is_politics'] == 0]
# df_not_health = df_not_health.sample(n=len(df_health),random_state=10)
# df_balanced = df_not_health.append(df_health)
# df_balanced.to_csv("balanced_univision.csv")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

import torch
from torch.utils.data import Dataset
# from omegaconf import DictConfig, OmegaConf
# import hydra
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
# import wandb
import os
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [ ]:
def compute_metrics(outs):
    predictions, labels = outs
    predictions = np.argmax(predictions, axis = -1)

    ## computes overall scores (accuracy, f1, recall, precision)
    accuracy = accuracy_score(labels, predictions) * 100
    f1 = f1_score(labels, predictions, average = "macro") * 100
    recall = recall_score(labels, predictions, average = "macro") * 100
    precision = precision_score(labels, predictions, average = "macro") * 100

    return {
        "accuracy" : float(accuracy),
        "f1" : float(f1),
        "recall" : float(recall),
        "precision" : float(precision),
    }



In [ ]:
def encode_labels(labels):
  labels_set = set(labels)
  endcoded_labels = labels
  # counter = 0
  # for current_label in labels_set:
  for j in range(len(endcoded_labels)):
    # print(endcoded_labels[j] )
    if endcoded_labels[j] == 'politica':
      endcoded_labels[j] = 1
    else:
      endcoded_labels[j] = 0
      # if endcoded_labels[j] == current_label:
      #   endcoded_labels[j] = counter
    # counter+=1
  return endcoded_labels

encode_labels(["x","health","y","x","z"])

In [ ]:
def load_data(path):
    """
    read CSV file and return the tweets and labels lists
    """
    df = pd.read_csv(path)
    titles = df['title'].tolist()
    labels = encode_labels(df['category'].tolist())
    print("max(labels)")

    print(max(labels))
    return titles, labels

In [ ]:
class MultiDialectDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels != None:
          item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
df = pd.read_csv("estadao_2010_2023.csv")
df = df[df['pubDate']>= str(2022)]
df = df[df['pubDate']< str(2022+1)]
df.category.value_counts()
df_health = df[df['category'] == 'politica']
df_health
# print(len(df_health))
max_len = len(df_health)
# print(max_len)
df_health = df_health.sample(n=max_len,random_state=10)
# print(len(df_health))

# print(len(df_not_health))
df_not_health = df[df['category'] != 'politica']

df_not_health = df_not_health.sample(n=max_len,random_state=10)
# print(len(df_not_health))

df_balanced = df_not_health.append(df_health)
df_balanced.to_csv("balanced_2022.csv")
# print(len(df_health))

# print(i)
print("------")
year_list = []
f1_list = []
for year in range(2006,2018):


  year_list.append(year)
  # df = pd.read_csv("balanced_2022.csv")
  # df
  val_tweets, val_labels = load_data("/content/drive/MyDrive/news_tagging_model/balanced_2022.csv")


  # print("Loading Model...")
  model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/news_tagging_model/microsoft_MiniLM-L12-H384-uncased_5epochs_"+str(year)+"-to-"+str(year+1)+"/best_ckpt", num_labels = 2)
  # print("Loading Tokenizer...")
  tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/news_tagging_model/microsoft_MiniLM-L12-H384-uncased_5epochs_2021/best_ckpt")



  # print("Tokenizeing the inputs...")
  # train_encodings = tokenizer(train_tweets,
  #                             truncation=True,
  #                             padding=True,
  #                             #max_length=model.config.max_position_embeddings
  #                             )
  val_encodings = tokenizer(val_tweets,
                            truncation=True,
                            padding=True,
                              #max_length=model.config.max_position_embeddings
                            )


  val_ds = MultiDialectDataset(val_encodings, val_labels)
  trainer = Trainer(model=model)
  trainer.model = model.cuda()
  val_pred = trainer.predict(val_ds)


  pred_0 = []
  pred_1 = []
  ispolitics_pred = []
  for pred in val_pred.predictions:
    pred_0.append(pred[0])
    pred_1.append(pred[1])
    if pred[0]>pred[1]:
      ispolitics_pred.append(0)
    else:
      ispolitics_pred.append(1)


  df_results = pd.DataFrame({'title':val_tweets,'ispolitics_truth':val_labels,'ispolitics_pred':ispolitics_pred,'pred_0':pred_0,'pred_1':pred_1})


  df_results.to_csv('Brazilian politics - estadao-export-2022-08-09 - prediction_output.csv')


  from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

  cm = confusion_matrix(val_labels, ispolitics_pred)

  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()
 
  f1 = f1_score(val_labels, ispolitics_pred)
  print("f1 score for year = "+str(year))
  print(f1)
  f1_list.append(f1)




In [ ]:
for i in range(len(f1_list)):
  print(str(year_list[i])+'-->'+str(f1_list[i]))

In [ ]:
df = pd.read_csv("Brazilian politics - estadao-export-2022-08-09 - translated.csv")
df =  df[df.Politics_Label.isin(['FALSE','TRUE'])]
val_tweets = df.title_es.tolist()
val_labels = []
for i in  df.Politics_Label.tolist():
  if i == 'FALSE':
    val_labels.append(0)
  elif i == 'TRUE':
    val_labels.append(1)
# val_labels.replace('FALSE',0)


year_list = []
f1_list = []
for year in range(2006,2018):
  # print("Loading Model...")
  model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/news_tagging_model/microsoft_MiniLM-L12-H384-uncased_5epochs_"+str(year)+"-to-"+str(year+1)+"/best_ckpt", num_labels = 2)
  # print("Loading Tokenizer...")
  tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/news_tagging_model/microsoft_MiniLM-L12-H384-uncased_5epochs_2021/best_ckpt")

  val_encodings = tokenizer(val_tweets,
                            truncation=True,
                            padding=True,
                              #max_length=model.config.max_position_embeddings
                            )

  val_ds = MultiDialectDataset(val_encodings, val_labels)
  trainer = Trainer(model=model)
  val_pred = trainer.predict(val_ds)

  pred_0 = []
  pred_1 = []
  ispolitics_pred = []
  for pred in val_pred.predictions:
    pred_0.append(pred[0])
    pred_1.append(pred[1])
    if pred[0]>pred[1]:
      ispolitics_pred.append(0)
    else:
      ispolitics_pred.append(1)

  df_results = pd.DataFrame({'title':val_tweets,'ispolitics_truth':val_labels,'ispolitics_pred':ispolitics_pred,'pred_0':pred_0,'pred_1':pred_1})

  cm = confusion_matrix(val_labels, ispolitics_pred)

  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()

  f1 = f1_score(val_labels, ispolitics_pred)
  print("f1 score for year = "+str(year))
  print(f1)
  f1_list.append(f1)



In [ ]:
for i in range(len(f1_list)):
  print(str(f1_list[len(f1_list)-i-1]))
  # print(str(year_list[i])+'-->'+str(f1_list[i]))

In [ ]:
# val_tweets = df.title.tolist()
# val_labels = df.is_politics.tolist()
# # for i in  df.Politics_Label.tolist():
# #   if i == 'FALSE':
# #     val_labels.append(0)
# #   elif i == 'TRUE':
# #     val_labels.append(1)
# # # val_labels.replace('FALSE',0)

In [ ]:
# train_all_tweets, train_all_labels = load_data("/content/drive/MyDrive/news_tagging_model/Brazilian politics - estadao-export-2022-08-09.csv")
# test_tweets, test_labels = load_data("/content/drive/MyDrive/news_tagging_model/Brazilian politics - estadao-export-2022-08-09.csv")

# #split the train_all to train and validation
# train_tweets, val_tweets, train_labels, val_labels = train_test_split(
#     train_all_tweets,
#     train_all_labels,
#     test_size=.99,
#     random_state= 5)

#tokenize the data


In [ ]:
# trainer.model = model.cuda()
# y = trainer.predict(small_eval_dataset

In [ ]:
2018 -> 0.8413999479573251 - 0.7377049180327869
2019 -> 0.8595555555555555 - 0.7301587301587301
2020 -> 0.8690936292232889 - 0.7000000000000001
2021 -> 0.8715429114826848 - 0.6861313868613138